In [5]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

In [2]:
path_10kLM = "/Users/apple/PROJECT/hons_project/data/SP500/LM/SEC/lm_sent_SEC_10k.parquet"
path_10qLM = "/Users/apple/PROJECT/hons_project/data/SP500/LM/SEC/lm_sent_SEC_10q.parquet"
path_SEC = "/Users/apple/PROJECT/hons_project/data/SP500/SEC/SEC_DTM_SP500.parquet"
path_SEC_LM = "/Users/apple/PROJECT/hons_project/data/SP500/LM/SEC/lm_sent_SEC_SP500.parquet"
df_10kLM = pd.read_parquet(path_10kLM)
df_10qLM = pd.read_parquet(path_10qLM)
df_SEC = pd.read_parquet(path_SEC)
df_SEC_LM = pd.read_parquet(path_SEC_LM)


In [3]:
df_all = pd.read_parquet(f'./data/SP500/SEC/SEC_DTM_SP500.parquet')
df_all = df_all.set_index('Date')
df_all.index = pd.to_datetime(df_all.index)
df_all['_ret'] = df_all['_ret']/100
print(df_all.shape)
lm_sent = pd.read_parquet(f'./data/SP500/LM/SEC/lm_sent_SEC_test2.parquet')
lm_sent = lm_sent.set_index('Date')
lm_sent.index = pd.to_datetime(lm_sent.index)
print(lm_sent.shape)

(25925, 14136)
(25925, 2)


In [ ]:
input_path = "./data/SP500/SEC/SEC_DTM_SP500.parquet"
dataset = pq.ParquetDataset(input_path)
table = dataset.read()
df_all = table.to_pandas()
df_all = df_all.reset_index()
df_all = df_all.sort_values(by=['Date', '_cik']).reset_index(drop=True)
df_all = df_all.drop(columns=["level_0"], errors='ignore')
df_all = df_all.set_index('Date')
df_all.index = pd.to_datetime(df_all.index)
df_all['_ret'] = df_all['_ret']/100



input_path = "./data/SP500/LM/SEC/lm_sent_SEC_test2.parquet"
dataset = pq.ParquetDataset(input_path)
table = dataset.read()
lm_sent = table.to_pandas()
lm_sent = lm_sent.reset_index()
lm_sent = lm_sent.sort_values(by=['Date', '_cik']).reset_index(drop=True)
lm_sent = lm_sent.drop(columns=["index"], errors='ignore')
lm_sent = lm_sent.set_index('Date')
lm_sent.index = pd.to_datetime(lm_sent.index)



In [24]:
# Sort lm_sent by _cik column and then by Date index
lm_sent = lm_sent.sort_values(by='_cik').sort_index()

# Sort df_all by _cik column and then by Date index
df_all = df_all.sort_values(by='_cik').sort_index()

# Print shapes and DataFrames to verify
print("lm_sent shape:", lm_sent.shape)
print("lm_sent DataFrame:")
print(lm_sent)

print("\ndf_all shape:", df_all.shape)
print("df_all DataFrame:")
print(df_all)

# Check if the indices and _cik columns are aligned
assert all(lm_sent.index == df_all.index) and all(lm_sent['_cik'] == df_all['_cik'])

lm_sent shape: (25925, 2)
lm_sent DataFrame:
                 _lm        _cik
Date                            
2006-01-12 -0.020383  0000936395
2006-01-12 -0.020386  0000936395
2006-01-17 -0.010600  0001090872
2006-01-17 -0.010570  0001090872
2006-01-27 -0.008370  0000063754
...              ...         ...
2024-11-13 -0.031473  0001746466
2024-11-26 -0.027870  0001140859
2024-12-05 -0.032044  0001327811
2024-12-12  0.000000  0000794367
2024-12-13 -0.020541  0000047217

[25925 rows x 2 columns]

df_all shape: (25925, 14136)
df_all DataFrame:
                  _cik      _vol      _ret    _vol+1    _ret+1  united  state  \
Date                                                                            
2006-01-12  0000936395  0.000052 -0.000112  0.001005 -0.011662    29.0   36.0   
2006-01-12  0000936395  0.000030  0.000079  0.001005 -0.011662    29.0   36.0   
2006-01-17  0001090872  0.000076  0.000078  0.000427 -0.021115    16.0   27.0   
2006-01-17  0001090872  0.000026 -0.000100  0.0

In [ ]:
print(lm_sent.head())
print(df_all[].head())

                 _lm        _cik
Date                            
2006-01-12 -0.020386  0000936395
2006-01-12 -0.020383  0000936395
2006-01-17 -0.010570  0001090872
2006-01-17 -0.010600  0001090872
2006-01-27 -0.008370  0000063754
                  _cik      _vol      _ret    _vol+1    _ret+1  united  state  \
Date                                                                            
2006-01-12  0000936395  0.000030  0.000079  0.001005 -0.011662    29.0   36.0   
2006-01-12  0000936395  0.000052 -0.000112  0.001005 -0.011662    29.0   36.0   
2006-01-17  0001090872  0.000026 -0.000100  0.000427 -0.021115    16.0   27.0   
2006-01-17  0001090872  0.000076  0.000078  0.000427 -0.021115    16.0   27.0   
2006-01-27  0000063754  0.000050 -0.000074  0.000098  0.013271     5.0    7.0   

            security  exchange  commission  ...  sorption  scroll  hermetic  \
Date                                        ...                               
2006-01-12      47.0      44.0        17.0 

In [9]:
all(lm_sent.index == df_all.index)

True

In [12]:
all(lm_sent['_cik'] == df_all['_cik'])

False

In [18]:
mismatch_mask = lm_sent['_cik'] != df_all['_cik']
# Rows in lm_sent where _cik differs
mismatched_lm_sent = lm_sent[mismatch_mask]

# Rows in df_all where _cik differs
mismatched_df_all = df_all[mismatch_mask]

# Display the mismatched rows side by side
print("Mismatched rows in lm_sent:")
print(mismatched_lm_sent[['_cik']])  # Show only _cik for clarity
print("\nMismatched rows in df_all:")
print(mismatched_df_all[['_cik']])

Mismatched rows in lm_sent:
                  _cik
Date                  
2006-02-09  0000920760
2006-02-09  0000101829
2006-02-16  0000079879
2006-02-16  0000814453
2006-02-16  0000079879
...                ...
2024-11-12  0000915912
2024-11-12  0001024478
2024-11-12  0000032604
2024-11-13  0000046765
2024-11-13  0001403161

[22438 rows x 1 columns]

Mismatched rows in df_all:
                  _cik
Date                  
2006-02-09  0000101829
2006-02-09  0000920760
2006-02-16  0000814453
2006-02-16  0000079879
2006-02-16  0001113169
...                ...
2024-11-12  0000891103
2024-11-12  0001818794
2024-11-12  0001024478
2024-11-13  0001403161
2024-11-13  0000046765

[22438 rows x 1 columns]


In [20]:
print(lm_sent.groupby('Date').size())
print(df_all.groupby('Date').size())
all(lm_sent.groupby('Date').size() == df_all.groupby('Date').size())

Date
2006-01-12    2
2006-01-17    2
2006-01-27    2
2006-02-01    2
2006-02-07    2
             ..
2024-11-13    4
2024-11-26    1
2024-12-05    1
2024-12-12    1
2024-12-13    1
Length: 3169, dtype: int64
Date
2006-01-12    2
2006-01-17    2
2006-01-27    2
2006-02-01    2
2006-02-07    2
             ..
2024-11-13    4
2024-11-26    1
2024-12-05    1
2024-12-12    1
2024-12-13    1
Length: 3169, dtype: int64


True